# <center>RPA transferencia entre bancos<center>

## Importações

In [504]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

## Pegando o login do banco apartir de um arquivo .ENV

In [505]:
load_dotenv()

db_host1 = os.getenv('DB_HOST1')
db_database1 = os.getenv('DB_DATABASE1')
db_user1 = os.getenv('DB_USER1')
db_password1 = os.getenv('DB_PASSWORD1')
db_port1 = os.getenv('DB_PORT1')

db_database2 = os.getenv('DB_DATABASE2')

print("DB_HOST1:", os.getenv('DB_HOST1'))
print("DB_DATABASE1:", os.getenv('DB_DATABASE1'))
print("DB_USER1:", os.getenv('DB_USER1'))
print("DB_PASSWORD1:", os.getenv('DB_PASSWORD1'))
print("DB_PORT1:", os.getenv('DB_PORT1'))
print("DB_DATABASE2:", os.getenv('DB_DATABASE2'))

DB_HOST1: recoop-germinare-9764.h.aivencloud.com
DB_DATABASE1: BANCO_1ANO
DB_USER1: avnadmin
DB_PASSWORD1: AVNS_DAFAJWqxMl1ba9hBbcZ
DB_PORT1: 16983
DB_DATABASE2: defaultdb


## Se conectando no banco do 1ano

In [506]:
print('Começando===============================================')
try:
    # Conectar ao banco de dados
    conn_1ano = psycopg2.connect(
        host=db_host1,
        database=db_database1,
        user=db_user1,
        password=db_password1,
        port=db_port1
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

Começando===============================================
Conexão estabelecida com sucesso!


## Pegando as informações do banco de dados

In [507]:
try:
    cursor_1ano = conn_1ano.cursor()

    cursor_1ano.execute('''SELECT * FROM Cooperativa''')
    results = cursor_1ano.fetchall()
    
    # Criar DataFrame
    columns = [desc[0] for desc in cursor_1ano.description]
    df = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Cooperativa''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_cooperativa = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Endereco''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_endereco = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Leilao''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_leilao = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Produto''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_produto = pd.DataFrame(results, columns=columns)

    cursor_1ano.execute('''SELECT * FROM Imagens''')
    columns = [desc[0] for desc in cursor_1ano.description]
    results = cursor_1ano.fetchall()
    df_foto = pd.DataFrame(results, columns=columns)
    
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

## Se conectando no banco do 2ano

In [508]:
print('Começando===============================================')
try:
    # Conectar ao banco de dados
    conn = psycopg2.connect(
        host=db_host1,
        database=db_database2,
        user=db_user1,
        password=db_password1,
        port=db_port1
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

Começando===============================================
Conexão estabelecida com sucesso!


## Pegando as informações de login da cooperativa

In [509]:
display(df_cooperativa)

,cnpj,nome,email,senha,telefone,imagem,dados_status,status
0,23456789012345,Cooperativa A,coop_a@email.com,Coop#1234,21987654321,None,False,Ativo
1,34567890123456,Cooperativa B,coop_b@email.com,Coop#5678,22987654322,None,False,Ativo
2,23456789060000,Cooperativa NovaB,contato@cooperativab.com,senha@5678,21987654432,None,False,Ativo
3,12345678905678,Cooperativa NovaA1,contato@cooperativaa.com,senha@1234,11987695432,None,False,Inativo


In [510]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()
                        
    for i in range(len(df_cooperativa)):
        if df_cooperativa['dados_status'][i] == True:
            try:
                # Tenta inserir no banco de dados
                cursor.execute('CALL insert_cooperativa(%s, %s, %s, %s, %s)', 
                    (df_cooperativa['cnpj'][i], df_cooperativa['nome'][i], df_cooperativa['email'][i], df_cooperativa['senha'][i], 
                    df_cooperativa['status'][i]))
            
            except Exception as e:
                print(f'Erro: {e}')
                print(f"Fazendo Update!!!===========================")
                conn.rollback()

                try:
                    # Tenta fazer o update
                    cursor.execute('UPDATE cooperativa SET nome_cooperativa=%s, email_cooperativa=%s, senha_cooperativa=%s, status=%s where cnpj_cooperativa=%s',
                               (df_cooperativa['nome'][i], df_cooperativa['email'][i], df_cooperativa['senha'][i],df_cooperativa['status'][i], df_cooperativa['cnpj'][i]))

                    # Confirma o update
                    conn.commit()

                except Exception as e_update:
                    conn.rollback()
                    print(f"Erro no update: {e_update}")

    conn.commit()
    print("Deu Green")

except Exception as e:
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        for i in range(len(df_cooperativa)):
            cursor_1ano.execute('UPDATE cooperativa SET dados_status=%s where cnpj=%s',
                                (False, df_cooperativa['cnpj'][i]))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")

Deu Green


# Endereço

In [511]:
display(df_endereco)

,id,numero,complemento,cep,bairro,cidade,logradouro,estado,dados_status,status
0,14,100,Apto 101,12345-678,Centro,São Paulo,Rua Exemplo,SP,False,Ativo
1,15,200,None,23456-789,Jardim,Rio de Janeiro,Avenida Modelo,RJ,False,Ativo
2,16,300,Bloco B,34567-890,Industrial,Belo Horizonte,Praça Demo,MG,False,Ativo
3,18,456,Bloco A,87654-321,Bairro Alto,Rio de Janeiro,Rua Exemplo 2,RJ,False,Ativo
4,17,123,Apto 101,12345-678,Centro,Fartura,Rua Exemplo 1,SP,False,Inativo


In [512]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()

    for i in range(len(df_endereco)):
        if df_endereco['dados_status'][i] == True:  # Corrige a verificação do status
            id_endereco = int(df_endereco['id'][i])
            cidade = str(df_endereco['cidade'][i])
            logradouro = str(df_endereco['logradouro'][i])
            numero = int(df_endereco['numero'][i])
            status = str(df_endereco['status'][i])

            try:
                # Tenta inserir no banco de dados
                cursor.execute('CALL insert_endereco(%s::int, %s::varchar, %s::varchar, %s::int, %s::varchar)', 
                    (id_endereco, cidade, logradouro, numero, status))

            except Exception as e:
                # Caso o insert falhe, tenta fazer o update
                print(f"Falha ao inserir, tentando atualizar: {e}")
                conn.rollback()
                print('Update sendo feito============')
                try:
                    # Tenta fazer o update
                    cursor.execute('UPDATE endereco SET cidade=%s, rua=%s, numero=%s, status=%s WHERE id_endereco=%s',
                    (cidade, logradouro, numero, status, id_endereco))  
                    
                    # Confirma o update
                    conn.commit()

                except Exception as e_update:
                    conn.rollback()
                    print(f"Erro no update: {e_update}")
                            

    # Confirma as alterações
    conn.commit()
    print("Deu Green")

except Exception as e:
    # Faz rollback em caso de erro
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        # Atualiza o campo dados_status na outra tabela
        for i in range(len(df_endereco)):
            id_endereco = int(df_endereco['id'][i])
            # Atualiza o status se a inserção for bem-sucedida
            cursor_1ano.execute('UPDATE endereco SET dados_status=%s WHERE id=%s', 
                            (False, id_endereco))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")

Deu Green


## Produto

In [513]:
display(df_produto)
display(df_foto)

,id,material,peso,dados_status,status
0,14,Aço,1500.5,False,Ativo
1,15,Plástico,500.0,False,Ativo
2,16,Vidro,750.3,False,Ativo
3,18,Metal,300.0,False,Ativo
4,17,Vidro,500.0,False,inativo


,id,imagem,id_produto,dados_status,status
0,14,None,14,True,Ativo
1,15,None,15,True,Ativo
2,18,None,18,True,Ativo
3,1,None,1,True,Ativo
4,2,None,2,True,Ativo
5,3,None,3,True,Ativo
6,4,None,4,True,Ativo
7,5,None,5,True,Ativo
8,6,None,6,True,Ativo


In [514]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()

    for i in range(len(df_produto)):
        # Verifica se o status é True
        if df_produto['dados_status'][i] == True:
            try:
                cursor.execute('''CALL insert_produto (%s, %s, %s, %s, %s, %s)''', 
                    (int(df_produto['id'][i]), df_produto['material'][i], float(df_leilao['valor_inicial'][i]), 
                    float(df_produto['peso'][i]), df_foto['imagem'][i], 
                    df_produto['status'][i]))

                # Confirma a inserção
                conn.commit()

            except Exception as e:
                # Faz rollback após erro no insert
                conn.rollback()
                print(f"Erro no insert: {e.__class__.__name__}: {e}")
                print("Fazendo Update================") 

                try:
                    cursor.execute('''UPDATE produto SET tipo_produto=%s, valor_inicial_produto=%s, peso_produto=%s, 
                                    foto_produto=%s, status=%s WHERE id_produto=%s''',
                                    (df_produto['material'][i], float(df_leilao['valor_inicial'][i]), 
                                    float(df_produto['peso'][i]), df_foto['imagem'][i], df_produto['status'][i], int(df_produto['id'][i])))

                    # Confirma o update
                    conn.commit()

                except Exception as e_update:
                    conn.rollback()
                    print(f"Erro no update: {e.__class__.__name__}: {e_update}")

        print("Deu Green")

except Exception as e:
    # Faz rollback se ocorrer erro geral
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        # Atualiza o campo dados_status na outra tabela
        for i in range(len(df_produto)):
            cursor_1ano.execute('UPDATE produto SET dados_status=%s WHERE id=%s',
                                (False, int(df_produto['id'][i])))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")


Deu Green
Deu Green
Deu Green
Deu Green
Deu Green


# Leilão

In [515]:
display(df_leilao)

,id,data_inicio,data_fim,hora,detalhe,valor_inicial,id_endereco,id_produto,id_cooperativa,dados_status,status
0,17,2024-08-28,2024-09-01,12:50:00,Leilão de ...,1000.0,17,17,12345678905678,False,Ativo
1,20,2024-09-10,2024-09-15,12:51:00,Leilão de ... e ...,2000.0,18,18,23456789060000,False,Ativo
2,21,2024-09-10,2024-09-15,12:51:00,Leilão de ... e ...,2000.0,18,18,23456789060000,False,Ativo
3,18,2024-09-10,2024-09-15,12:51:00,Leilão de ... e ...,2000.0,18,18,23456789060000,False,Ativo
4,19,2024-08-28,2024-09-01,12:50:00,Leilão de Plastico,1000.0,17,17,12345678905678,True,inativo


In [516]:
try:
    cursor = conn.cursor()
    cursor_1ano = conn_1ano.cursor()

    for i in range(len(df_leilao)):
        if df_leilao['dados_status'][i] == True:
            try:
                # Tenta inserir no banco de dados
                cursor.execute('CALL insert_leilao(%s, %s, %s, %s, %s, %s, %s, %s, %s)', 
                    (int(df_leilao['id_endereco'][i]), df_leilao['id_cooperativa'][i], df_leilao['data_inicio'][i], 
                     df_leilao['data_fim'][i] ,df_leilao['detalhe'][i] ,int(df_leilao['id'][i]), df_leilao['hora'][i], 
                     int(df_leilao['id_produto'][i]),df_leilao['status'][i]))

                print('passei aqui')
            except Exception as e:
                print(f'Erro: {e}')
                print(f"Update feito==============")
                conn.rollback()

                try:
                    cursor.execute('UPDATE leilao SET data_inicio_leilao=%s, data_fim_leilao=%s, hora_leilao=%s, id_endereco=%s, detalhes_leilao=%s, id_produto=%s, status=%s WHERE id_leilao=%s', 
                    (df_leilao['data_inicio'][i], df_leilao['data_fim'][i], df_leilao['hora'][i], 
                     int(df_leilao['id_endereco'][i]), df_leilao['detalhe'][i], int(df_leilao['id_produto'][i]), df_leilao['status'][i],int(df_leilao['id'][i])))

                    conn_1ano.commit()
                except Exception as e:
                    print(f"Erro no update: {e}")
                    conn.rollback()

    # Confirma as alterações
    conn.commit()
    print("Deu Green")

except Exception as e:
    # Faz rollback em caso de erro
    conn.rollback()
    print(f"Erro ao executar: {e}")

finally:
    try:
        for i in range(len(df_leilao)):
            cursor_1ano.execute('UPDATE leilao SET dados_status=%s WHERE id=%s', 
                                (False, int(df_leilao['id'][i])))

        conn_1ano.commit()
    except Exception as e:
        print(f"Erro no finally ao tentar atualizar status: {e}")
    # Fechar o cursor e a conexão
    cursor.close()
    conn.close()
    cursor_1ano.close()
    conn_1ano.close()

Erro: duplicate key value violates unique constraint "leilao_pkey"
DETAIL:  Key (id_leilao)=(19) already exists.
CONTEXT:  SQL statement "INSERT INTO leilao (id_leilao,data_inicio_leilao, data_fim_leilao, detalhes_leilao, hora_leilao, id_endereco, cnpj_cooperativa, id_produto, status)
            VALUES (lan_id_leilao,leilao_data_inicio, leilao_data_fim, leilao_detalhes_leilao, lan_hora, lan_id_endereco, lan_cnpj_cooperativa, lan_id_produto, lan_status)"
PL/pgSQL function insert_leilao(integer,character varying,date,date,character varying,integer,time without time zone,integer,character varying) line 8 at SQL statement

Update feito==============
Deu Green


In [517]:
cursor.close()
conn.close()
cursor_1ano.close()
conn_1ano.close()